In [9]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
import datetime
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam

In [3]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar.load_data()

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='tanh', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='tanh'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='tanh'))
model.add(layers.Dense(10))

model.summary()

Metal device set to: Apple M1 Max
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)    

In [5]:
model.compile(optimizer=Adam(0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
# Clear any logs from previous runs
!rm -rf ./logs/fit_Tanh_Adam/

log_dir = "logs/fit_Tanh_Adam/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, )

history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[tensorboard_callback])

Epoch 1/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.9158 - accuracy: 0.6813 - val_loss: 1.0564 - val_accuracy: 0.6268
Epoch 2/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.8866 - accuracy: 0.6951 - val_loss: 1.0517 - val_accuracy: 0.6326
Epoch 3/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.8579 - accuracy: 0.7031 - val_loss: 1.0652 - val_accuracy: 0.6300
Epoch 4/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.8314 - accuracy: 0.7144 - val_loss: 1.0475 - val_accuracy: 0.6320
Epoch 5/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.8076 - accuracy: 0.7239 - val_loss: 1.0467 - val_accuracy: 0.6374
Epoch 6/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.7826 - accuracy: 0.7311 - val_loss: 1.0448 - val_accuracy: 0.6332
Epoch 7/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.7577 - accuracy: 0.7426 - val_loss: 1.0360 - val_ac

In [11]:
test_loss, test_acc = model.evaluate(x_test,  y_test)
print(test_acc)

313/313 [==============================] - 2s 6ms/step - loss: 1.0623 - accuracy: 0.6340
0.6340000033378601


In [12]:
#/gradient_tape
%tensorboard --logdir logs

Launching TensorBoard...